In [1]:
import netCDF4 as nc
import numpy as np
from pyproj import Proj, transform, Transformer

input_file = r"F:/Simulation_PALM/_simulation_data/_kn_4096x4096_base - backup/konstanz_4096x4096_v4_3d_N03.000.nc"
output_file = r"F:/reduced.nc"
# Read NetCDF file as dataset
ds = nc.Dataset(input_file, mode="r")

origin_x = ds.origin_x
origin_y = ds.origin_y

In [2]:
variable_list = ['x', 'y', 'xu', 'yv']
# for variable_name in variable_list:
x = ds['x'][:]
y = ds['y'][:]
xu = ds['xu'][:]
yv = ds['yv'][:]

coordinates_x = x + origin_x - 1
coordinates_y = y + origin_y - 1

transformer = Transformer.from_crs("EPSG:25832", "EPSG:4326") # Transformer.from_crs(source, target)
# Germany lat, lon = 51.16 N, 10.45 E

# AOI Child Boundary
xlist, y_list = [512902.0, 513414.0], [5278402.0, 5278914.0]

lat_values, lon_values = transformer.transform(coordinates_x, coordinates_y)
print(lat_values[:5])
print(lon_values[:5])

def band_time_n_hour(x_tick_interval):
    # create band and time sequence based on x_tick_interval
    band_sequence = [0]
    first_hour = 6*x_tick_interval - 1
    band_sequence.extend(np.arange(first_hour, 144, 6 * x_tick_interval))

    time_sequence = [((band_index+1) / 6) for band_index in band_sequence]
    time_sequence[0] = 0.1
    
    # define levels to export
    level_sequence = [4,6,8,10,12,16,18,20] # 0=4, 7=20
    return band_sequence, time_sequence, len(time_sequence), level_sequence

x_tick_interval = 3 # n hour
band_sequence, time_sequence, time_steps, level_sequence = band_time_n_hour(x_tick_interval)

[47.65888374 47.65890169 47.65891965 47.6589376  47.65895556]
[9.17182984 9.17185654 9.17188323 9.17190993 9.17193662]


In [3]:
useful_global_attributes = ["origin_time", "origin_lat", "origin_lon", "origin_x", "origin_y", "origin_z"]
variable_list = ['u', 'v']

with nc.Dataset(output_file, mode="w") as out_file:
        # Copy global attributes from the input file to the output file
        for attr_name in useful_global_attributes:
            if attr_name in ds.ncattrs():
                out_file.setncattr(attr_name, ds.getncattr(attr_name))
        
        # Create dimensions wrt the original dataset (time, zu_3d, y, x)
        for dim_name, dim in ds.dimensions.items():
            if dim_name == 'time':
                out_file.createDimension(dim_name, time_steps)
                out_variable = out_file.createVariable(dim_name, ds[dim_name].dtype, (dim_name,))
                out_variable[:] = time_sequence
            elif dim_name == 'zu_3d':
                new_dim_name = 'lev'
                out_file.createDimension(new_dim_name, len(level_sequence))
                out_variable = out_file.createVariable(new_dim_name, ds[dim_name].dtype, (new_dim_name,))
                out_variable[:] = level_sequence
            elif dim_name == 'y':
                new_dim_name = 'lat'
                out_file.createDimension(new_dim_name, len(ds[dim_name]))
                out_variable = out_file.createVariable(new_dim_name, ds[dim_name].dtype, (new_dim_name,))
                out_variable[:] = lat_values
            elif dim_name == 'x':
                new_dim_name = 'lon'
                out_file.createDimension(new_dim_name, len(ds[dim_name]))
                out_variable = out_file.createVariable(new_dim_name, ds[dim_name].dtype, (new_dim_name,))
                out_variable[:] = lon_values
                
        # Create variable and copy data as required
        for variable_name in variable_list:
            out_variable = out_file.createVariable(variable_name, ds[variable_name].dtype, ('time', 'lev', 'lat', 'lon'))
            for i, band_index in enumerate(band_sequence):
                # Extract the subset of data and save it to the output variable
                for j, level in enumerate(level_sequence):
                    band_data = ds[variable_name][band_index, level, :, :]
                    out_variable[i, j, :, :] = band_data


In [4]:
output_file = r"F:/reduced.nc"
# Read NetCDF file as dataset
with nc.Dataset(output_file, mode="r") as ds_reduced:
    # print(ds_reduced)
    print(ds_reduced['lat'][:5])
    print(ds_reduced['lon'][:5])

# sample_file = r"F:\wind_global.nc"
# with nc.Dataset(sample_file, mode="r") as sample_file:
#     print(sample_file['u_wind'][:10,:])


[47.65888374 47.65890169 47.65891965 47.6589376  47.65895556]
[9.17182984 9.17185654 9.17188323 9.17190993 9.17193662]


In [22]:
import leafmap
file = r"F:/wind_global.nc"
netcdf_data = leafmap.read_netcdf(file)
m = leafmap.Map(width=500, height=500, zoom=4)
m.set_center(lon_lat[0], lon_lat[1])
m.add_basemap('Google Satellite')
m.add_velocity(netcdf_data,
                   zonal_speed='u_wind',
                   meridional_speed='v_wind',
                #    latitude_dimension='lat',
                   velocity_scale=0.012
                   )
m


Map(center=[47.65978150120808, 9.173164618098161], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [18]:
import leafmap

netcdf_data = leafmap.read_netcdf(output_file)
lon_lat = [9.11595592, 47.46664243] #x_y

with nc.Dataset(output_file, mode="r") as ds_reduced:
    test = (ds_reduced['lon'][50], ds_reduced['lat'][50])
    lon_lat = (test[0].item(), test[1].item())
    
m = leafmap.Map(width=850, height=800, zoom=16)
m.set_center(lon_lat[0], lon_lat[1])
m.add_basemap('Google Satellite')
m.add_velocity(netcdf_data,
                   zonal_speed='u',
                   meridional_speed='v',
                   latitude_dimension='lat',
                   longitude_dimension='lon',
                   level_dimension='lev',
                   level_index = 1,
                   time_index=6,
                #    velocity_scale=0.02,
                   display_options= dict({
                       'speedUnit' : 'm/s',
                       'displayEmptyString': 'No velocity data',
                       }),
                   max_velocity=3,
                   name=f"Wind",
                   )
m

Map(center=[47.65978150120808, 9.173164618098161], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [29]:
a = '12:00'

time = a.replace(":","")
print(time)

print(type(time))

1200
<class 'str'>
